# Train and Apply Models

In [1]:
from ML.model_training import (
    omit_patient_video,
    single_user_split,
    train_lstm,
    train_random_forest
)
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    confusion_matrix,
    classification_report,
)
import numpy as np
import pandas as pd
from ML import utils
import sys
import random
from itertools import product

In [10]:
# remove_list = [0, 1, 2, 4, 5, 6, 8, 9, 10, 11, 14, 16, 17, 18, 19, 22]
remove_list=[20]
X_train_vals, X_test_vals, arousal_train_vals, arousal_test_vals = omit_patient_video(
    target="arousal",
    trials=5,
    # selected_user=3,
    exclude_users=remove_list,
    filename="datasets/features_table_psd_only.csv"
)
features = utils.filter_features(X_train_vals.columns, remove_bands=["gamma", "delta"])

def balance(X, y, seed=5):
    c = y.value_counts()
    if c.get("high", 0) == c.get("low", 0):
        return X.reset_index(drop=True), y.reset_index(drop=True)
    maj = c.idxmax()
    m = c.min()
    keep = y[y != maj].index.union(y[y == maj].sample(m, random_state=seed).index)
    return X.loc[keep].reset_index(drop=True), y.loc[keep].reset_index(drop=True)

arousal_train = pd.Series(
    np.where(arousal_train_vals > 3.8, "high", "low"),
    index=arousal_train_vals.index,
    dtype="string",
)
arousal_test = pd.Series(
    np.where(arousal_test_vals > 3.8, "high", "low"),
    index=arousal_test_vals.index,
    dtype="string",
)

# X_train, arousal_train = balance(X_train_vals, arousal_train)
# X_test, arousal_test = balance(X_test_vals, arousal_test)

print("arousal_train counts:\n", arousal_train.value_counts(dropna=False))
print("arousal_test counts:\n", arousal_test.value_counts(dropna=False))

subjects = [1, 3, 12, 15]
# subjects = []
# for _ in range(5):
#     subjects.append(random.randint(0, 22))
# print(subjects)

Held-out patient: 15 | Held-out (patient, video) trials: [(15, 5), (15, 6), (15, 8), (15, 12), (15, 17)] | Excluded users: [20]
arousal_train counts:
 low     21401
high    18610
Name: count, dtype: Int64
arousal_test counts:
 high    464
low      71
Name: count, dtype: Int64


## LSTM CV Optimizer

In [5]:


param_grid = {
    "lr": [0.0001],
    "epochs": [25],
    "units": [128],
    "batch_size": [128],
}

best_params = None
best_mean_acc = -float("inf")

print("Starting global hyperparameter search...\n")

for lr, epochs, units, batch_size in product(
    param_grid["lr"],
    param_grid["epochs"],
    param_grid["units"],
    param_grid["batch_size"],
):
    combo_accs = []
    for i in subjects:
        relabel = False
        while True:
            while True:
                X_train, X_test, arousal_train, arousal_test = omit_patient_video(
                    target="arousal",
                    selected_user=i,
                    trials=18,
                    # holdout_videos=[2, 10, 15],
                    exclude_users=remove_list,
                    filename="datasets/features_table_psd_only.csv",
                    relabel=relabel,
                )
            

                # binarize labels
                arousal_train = pd.Series(
                    np.where(arousal_train > 3.8, "high", "low"),
                    index=arousal_train.index,
                    dtype="string",
                )
                arousal_test = pd.Series(
                    np.where(arousal_test > 3.8, "high", "low"),
                    index=arousal_test.index,
                    dtype="string",
                )

                # optional class sanity check if you want it back:
                # c = arousal_test.value_counts()
                # if c.get("high", 0) == 0 or c.get("low", 0) == 0:
                #     continue

                # X_train, arousal_train = balance(X_train, arousal_train)
                # X_test, arousal_test = balance(X_test, arousal_test)
                print("arousal_train counts:\n", arousal_train.value_counts(dropna=False))
                print("arousal_test counts:\n", arousal_test.value_counts(dropna=False))
                break

            X_train_sub = X_train.loc[:, features]
            X_test_sub = X_test.loc[:, features]

            lstm, X_test_eval, y_test_eval = train_lstm(
                X_train_sub,
                X_test_sub,
                arousal_train,
                arousal_test,
                lr=lr,
                epochs=epochs,
                units=units,
                batch_size=batch_size,
                dropout=0.5,
                recurrent_dropout=0.5,
                bidirectional=True,
            )
            y_prob = lstm.predict(X_test_eval).ravel()
            arousal_pred = (y_prob >= 0.5).astype(int)

            acc = accuracy_score(y_test_eval, arousal_pred)
            print(acc)
            if acc < 0.55 and not relabel:
                print("attempt relabel")
                relabel = True
            else:
                relabel = False
                break

        combo_accs.append(float(acc))

        print("\nConfusion Matrix (subject):")
        print(confusion_matrix(y_test_eval, arousal_pred))
    mean_acc = float(np.mean(combo_accs))
    print(
        f"Params lr={lr}, epochs={epochs}, units={units}, batch_size={batch_size} "
        f"-> mean acc across subjects = {mean_acc:.4f}"
    )

    if mean_acc > best_mean_acc:
        best_lstm = lstm
        best_mean_acc = mean_acc
        best_params = {
            "lr": lr,
            "epochs": epochs,
            "units": units,
            "batch_size": batch_size,
        }

print("\nBest universal hyperparameters:")
print(best_params)
print(f"Mean accuracy across subjects (tuning): {best_mean_acc:.4f}\n")

Starting global hyperparameter search...

Held-out patient: 1 | Held-out (patient, video) trials: [(1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17)] | Excluded users: []
arousal_train counts:
 low     21273
high    19273
Name: count, dtype: Int64
arousal_test counts:
 high    1071
low      772
Name: count, dtype: Int64
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step 
0.46066196418882255
attempt relabel
Held-out patient: 1 | Held-out (patient, video) trials: [(1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17)] | Excluded users: []
arousal_train counts:
 high    24112
low     16434
Name: count, dtype: Int64
arousal_test counts:
 high    1096
low      747
Name: count, dtype: Int64
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step 
0.594139989148128

Confusion Matrix (subject):
[[  10  737]
 [  11 1085]]
Held-o

### Fine-tune for User

In [ ]:
from tensorflow.keras.models import clone_model
from tensorflow.keras import optimizers
import tensorflow as tf
import numpy as np
import pandas as pd

df = utils.read_table("datasets/features_table_psd_only.csv").reset_index(drop=True)

df = df[~df["patient_index"].isin(remove_list)].reset_index(drop=True)

X = df.drop(
    columns=["patient_index", "video_index", "arousal", "valence", "Unnamed: 0"],
    errors="ignore",
)
X = X.sort_index(axis=1)

arousal_target = df["arousal"].astype(float)

features = utils.filter_features(X.columns, remove_bands=["gamma", "delta"])
X = X.loc[:, features]

print("Global X shape:", X.shape)

y_global = (arousal_target > 3.8).astype("int32")

best_lstm, X_test_eval, y_test_eval = train_lstm(
    X,
    None,
    y_global,
    None,
    lr=0.0001,
    epochs=25,
    units=128,
    batch_size=128,
    bidirectional=True,
)

X_user_train, X_user_test, arousal_user_train, arousal_user_test = single_user_split(
    target="arousal",
    selected_user=20,
    k_holdouts=5,
    holdout_videos=[2, 7, 10, 15, 17],
    filename="datasets/features_table_psd_only.csv",
)

arousal_user_train = pd.Series(arousal_user_train, dtype="float32")
arousal_user_test  = pd.Series(arousal_user_test, dtype="float32")


y_user_train = (arousal_user_train > 3.8).astype("int32").to_numpy()
y_user_test  = (arousal_user_test  > 3.8).astype("int32").to_numpy()
print("arousal_train counts:\n", arousal_user_train.value_counts(dropna=False))
print("arousal_test counts:\n", arousal_user_test.value_counts(dropna=False))

X_user_train = X_user_train.sort_index(axis=1)
X_user_test  = X_user_test.sort_index(axis=1)

X_user_train = X_user_train.loc[:, features]
X_user_test  = X_user_test.loc[:, features]

X_user_train = X_user_train.to_numpy(dtype="float32")
X_user_test  = X_user_test.to_numpy(dtype="float32")

X_user_train = X_user_train.reshape(-1, 1, X_user_train.shape[1])
X_user_test  = X_user_test.reshape(-1, 1, X_user_test.shape[1])

print("User train shape:", X_user_train.shape)
print("User test shape:", X_user_test.shape)

user_model = clone_model(best_lstm)
user_model.build(best_lstm.input_shape)
user_model.set_weights(best_lstm.get_weights())

# Freeze all but last 2 layers (optional)
for layer in user_model.layers[:-2]:
    layer.trainable = False

# If best_lstm already uses 1-unit sigmoid output, this is correct:
user_model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

history = user_model.fit(
    X_user_train,
    y_user_train,
    validation_split=0.15,
    batch_size=32,
    epochs=10,
    callbacks=[tf.keras.callbacks.EarlyStopping(
        patience=3, restore_best_weights=True, monitor="val_loss"
    )],
    verbose=1,
)


test_loss, test_acc = user_model.evaluate(X_user_test, y_user_test, verbose=0)
print(f"\nUser Test accuracy: {test_acc:.4f}")

# For 1-unit sigmoid output:
y_prob = user_model.predict(X_user_test).ravel()
y_pred = (y_prob >= 0.5).astype("int32")

cm = confusion_matrix(y_user_test, y_pred, labels=[0, 1])
print("\nConfusion Matrix (rows=true, cols=pred):")
print(cm)

print("\nOverall accuracy:", accuracy_score(y_user_test, y_pred))

print("\nClassification report:")
print(classification_report(y_user_test, y_pred, target_names=["low", "high"]))


Global X shape: (40546, 98)
